In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pickle
import os
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from importlib import reload
from trecs.metrics import MSEMeasurement, InteractionSpread, InteractionSpread, InteractionSimilarity, RecSimilarity, RMSEMeasurement, InteractionMeasurement
from trecs.components import Users
import trecs.matrix_ops as mo
import src.globals as globals
import seaborn as sns

from wrapper.models.bubble import BubbleBurster
from src.utils import *
from src.plotting import plot_measurements 
from src.scoring_functions import cosine_sim, entropy, content_fairness
from wrapper.metrics.evaluation_metrics import *

random_state = np.random.seed(42)
plt.style.use("seaborn")

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

globals.initialize()

# RecommenderSystem on MovieLens 

In [2]:
n_attrs=20
max_iter=1000
n_clusters=25

In [3]:
score_fn = ''
probabilistic = False
globals.ALPHA = 0.2
alpha = globals.ALPHA

# User parameters
drift = 0.05
attention_exp=-0.8

## Embeddings

In [4]:
binary_ratings_matrix = load_and_process_movielens(file_path='data/ml-100k/u.data')

In [5]:
# Get user and item representations using NMF
user_representation, item_representation = create_embeddings(binary_ratings_matrix, n_attrs=n_attrs, max_iter=max_iter)

Loaded embeddings.


In [6]:
# Define topic clusters using NMF
item_cluster_ids, item_cluster_centers = get_clusters(item_representation.T, name='item', n_clusters=n_clusters, n_attrs=n_attrs, max_iter=max_iter)
user_cluster_ids, user_cluster_centers = get_clusters(user_representation, name='user', n_clusters=n_clusters, n_attrs=n_attrs, max_iter=max_iter)

Loaded clusters.
Loaded clusters.


### Model

In [7]:
num_users = len(user_representation)
num_items = len(item_representation)
print(f'Number of items: {num_items}')
print(f'Number of users: {num_users}')

users = Users(actual_user_profiles=user_representation, 
              repeat_interactions=False, 
              drift=drift,
              attention_exp=attention_exp)

Number of items: 20
Number of users: 943


In [8]:
# Create user_pairs by pairing users only with others that are not in the same cluster
num_users = len(user_representation)
inter_cluster_user_pairs = []
for u_idx in range(num_users):
    for v_idx in range(num_users):
        if user_cluster_ids[u_idx] != user_cluster_ids[v_idx]:
            inter_cluster_user_pairs.append((u_idx, v_idx))

intra_cluster_user_pairs = []
for u_idx in range(num_users):
    for v_idx in range(num_users):
        if user_cluster_ids[u_idx] == user_cluster_ids[v_idx]:
            intra_cluster_user_pairs.append((u_idx, v_idx))

In [9]:
mse = MSEMeasurement()
measurements = [
    InteractionMeasurement(),
    MSEMeasurement(),  
    InteractionSpread(),                
    InteractionSimilarity(pairs=inter_cluster_user_pairs, name='inter_cluster_interaction_similarity'), 
    InteractionSimilarity(pairs=intra_cluster_user_pairs, name='intra_cluster_interaction_similarity'), 
    RecSimilarity(pairs=inter_cluster_user_pairs, name='inter_cluster_rec_similarity'), 
    RecSimilarity(pairs=intra_cluster_user_pairs, name='intra_cluster_rec_similarity'), 
    UserMSEMeasurement(),
    SerendipityMetric(), 
    DiversityMetric(), 
    NoveltyMetric(),
    RecallMeasurement(),
    MeanNumberOfTopics(),
]

In [10]:
# Model
config = {
    'actual_user_representation': users,
    'actual_item_representation': item_representation,
    'item_topics': item_cluster_ids,
    'num_attributes': n_attrs,
    'num_items_per_iter': 10,
    'seed': 42,
    'record_base_state': True,
}

model_name='myopic'
requires_alpha = False

if score_fn:
    if score_fn == 'cosine_sim':
        config['score_fn'] = cosine_sim
        requires_alpha = True
    elif score_fn == 'entropy':
        config['score_fn'] = entropy
        requires_alpha = True
    elif score_fn == 'content_fairness':
        config['score_fn'] = content_fairness        
    else:
        raise Exception('Given score function does not exist.')
    model_name = score_fn

if probabilistic:
    config['probabilistic_recommendations'] = True
    model_name += '_prob'

In [11]:
model = BubbleBurster(**config)

model.add_metrics(*measurements)

## Simulation

In [12]:
# Fair Model
train_timesteps=5
model.startup_and_train(timesteps=train_timesteps)

100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:46<00:00,  9.39s/it]


In [ ]:
run_timesteps=20
model.run(timesteps=run_timesteps)

 20%|████████████████▊                                                                   | 4/20 [00:49<03:20, 12.52s/it]

In [ ]:
import src
reload(src.utils)
from src.utils import *
    
# Determine file name based on parameter values
parameters = f'_{train_timesteps}trainTimesteps_{run_timesteps}runTimesteps_{n_attrs}nAttrs_{n_clusters}nClusters_{drift}Drift_{attention_exp}AttentionExp'
if requires_alpha:
    parameters += f'_{alpha}Lambda'

# Save actual user preferences
final_preferences_dir = 'artefacts/final_preferences/'
file_prefix = f'{model_name}_final_preferences'
final_preferences_path = final_preferences_dir + file_prefix + parameters + '.npy'
np.save(final_preferences_path, model.users.actual_user_profiles.value, allow_pickle=True)

# Save measurements
measurements_dir = f'artefacts/measurements/'
file_prefix = f'{model_name}_measurements'

measurements_path = measurements_dir + file_prefix + parameters + '.csv'
measurements_df = load_or_create_measurements_df(model, model_name, train_timesteps, measurements_path)
measurements_df.to_csv(measurements_path)
print('Measurements saved.')

## Analysis

In [ ]:
# Create df for parameters
numeric_cols = ['trainTimesteps', 'runTimesteps', 'nAttrs', 'nClusters', 'Lambda']
columns = ['model_name'] + numeric_cols

data = [[model_name, train_timesteps, run_timesteps, n_attrs, n_clusters, None]]
if requires_alpha:
    data = [[model_name, train_timesteps, run_timesteps, n_attrs, n_clusters, alpha]]

parameters_df = pd.DataFrame(data,
                             columns = columns)
for col in numeric_cols:
    parameters_df[col] = pd.to_numeric(parameters_df[col])

In [ ]:
plot_measurements([measurements_df], parameters_df)

# Experiments

In [ ]:
recommender = model

In [ ]:
# Novelty that is based on probability calculation from generate_recommendations 
# (leave previously interacted items out)
item_counts = recommender.item_count
item_counts[item_counts == 0] = 1
items_self_info = (-1) * np.log(item_counts)

# turn scores in probability distribution over items to ensure that all independent of the ranking function, the metric yields comparable values
scores = recommender.predicted_scores.value
probabilities = scores / np.sum(scores, axis=1)[:, np.newaxis]

item_indices = recommender.indices
if not recommender.users.repeat_interactions:
    # for each user, eliminate items that have been interacted with
    item_indices = item_indices[np.where(item_indices >= 0)]
    item_indices = item_indices.reshape((recommender.num_users, -1))

s_filtered = mo.to_dense(recommender.predicted_scores.filter_by_index(item_indices))
row = np.repeat(recommender.users.user_vector, item_indices.shape[1])
row = row.reshape((recommender.num_users, -1))
permutation = s_filtered.argsort()
rec = item_indices[row, permutation]
# the recommended items will not be exactly determined by
# predicted score; instead, we will sample from the sorted list
# such that higher-preference items get more probability mass
num_items_unseen = rec.shape[1]  # number of items unseen per user
probabilities = np.logspace(0.0, num_items_unseen / 10.0, num=num_items_unseen, base=2)
probabilities = probabilities / probabilities.sum()

# get utility of each item given a state of users
item_states = np.mean(probabilities, axis=0)

# calculate novelty per item by multiplying self information and utility value
item_novelties = items_self_info * item_states
# form sum over all possible items/actions
item_novelty = np.sum(item_novelties)

item_novelty